In [5]:
import sys

import numpy as np
import pandas as pd
import mdptoolbox as mtb

import itertools as it
import pprint as pp
from scipy.stats import binom

In [6]:
np.set_printoptions(threshold=sys.maxsize)

In [7]:
# NOTATION

# rplant - renewable plant 
# fplant - fossil fuel plant
# RES - referring to renewable plant
# FF - referring to fossil fuel plant

In [49]:
# VARIABLES

N_YEARS = 30
N_TECHSTAGES = 3
N_PLANTS = 5

PLANT_SIZE = 600000
CAPACITY = 0.35
HOURS_YR = 8736

# Starting price of carbon per ton.
C_CO2_INIT = 40
# Initial construction costs of a renewable plant per kW (average of solar PV and onshore wind).
C_CAP_RES = [1284, 746, 456]
# Annual operation & maintenance costs of a fossil fuel plant per kW (average of coal and natural gas).
C_OM_FF = 68.8
# Annual emissions of a fossil fuel plant in tons CO2 per kWh (average of coal and natural gas).
FF_EMIT = 0.0011

# # Power output (size) * capacity factor * hours per year
# POWER_FF = 600000*0.3*8736

# Probability that tech stage advances to the next given the current stage is not the highest. 
# Assume it is only possible to advance by 1 at a time.
P_ADV_TECHSTAGE = 0.25

# Probability that a renewable plant "fails" at the end of the year.
# A plant that fails is replaced in the next year for the same cost as building a new plant.
RPLANT_LIFE = 25
P_RPLANT_FAIL = 1 / RPLANT_LIFE

# Discount rate (average of solar PV and onshore wind in North America).
DISC_RATE = 0.06

In [9]:
# STATE SPACE

# State space includes:
    # T = Time 
        # Range 2020 to 2050 years
    # V = Tech "stage" 
        # Represents how advanced current energy technologies are
    # N_r = Number of renewable power plants 
        # Out of N total plants
        # Number of fossil fuel plants is N - N_r
        
S = (N_YEARS+1) * (N_TECHSTAGES) * (N_PLANTS+1)

# Create mapping between state and unique integer ID.
def enumerate_states(n_years, n_techstages, n_plants):
    state_to_id = {}
    idx = 0
    iter_states = get_iter_states(n_years, n_techstages, n_plants)
    for state in iter_states:
        (t, v, r) = state
        state_to_id[state] = idx
        idx += 1
    return state_to_id

In [12]:
state_to_id = enumerate_states(N_YEARS, N_TECHSTAGES, N_PLANTS)
id_to_state = {v: k for k, v in state_to_id.items()}
#pp.pprint(state_to_id)

In [13]:
# ACTION SPACE

# Possible actions:
    # 0 -- Do nothing
    # 1...N -- Convert 1...N fossil fuel plants to renewable plants
# An invalid action attempts to convert more fossil fuel plants than remain. 
    
A = N_PLANTS + 1

In [17]:
# TRANSITION PROBABILITIES

def trans_probs_wrapper(A, S):
    transitions = np.zeros([A, S, S])
    print("Filling transitions probabilities for A = 0 (do nothing)...")
    fill_trans_donothing(transitions)
    print("Filling transitions probabilities for other A...")
    fill_trans_other(transitions)
    print("Transitions done.")
    return transitions

def fill_trans_donothing(transitions):
    iter_states = get_iter_states(N_YEARS, N_TECHSTAGES, N_PLANTS)
    for state in iter_states:
        (t, v, r), state_curr, idx_curr = breakdown_state(state)
        # Edge case for terminal state.
        if t == N_YEARS:
            transitions[0][idx_curr][idx_curr] = 1.0
            continue
        # FAILURE LOOP
        loop_failure(state_curr, transitions, 0, 0)
        assert np.isclose(np.sum(transitions[0][idx_curr]), 1.0), np.sum(transitions[0][idx_curr])

def fill_trans_other(transitions):
    iter_states = get_iter_states(N_YEARS, N_TECHSTAGES, N_PLANTS)
    for state in iter_states:
        (t, v, r), state_curr, idx_curr = breakdown_state(state)
        # ACTION LOOP
        # From 1 up to number of fossil fuel plants remaining may be converted.
        for a in np.arange(1, A):
            # Transition doesn't matter for final year as long as it exists. 
            if t == N_YEARS:
                transitions[a][idx_curr][idx_curr] = 1.0
                continue
            if a > N_PLANTS - r:
                # For invalid actions build the max plants possible. 
                loop_failure(state_curr, transitions, N_PLANTS-r, a)
            else:
                # FAILURE LOOP
                loop_failure(state_curr, transitions, a, a)
            assert np.isclose(np.sum(transitions[a][idx_curr]), 1.0), np.sum(transitions[a][idx_curr])
            normalize_trans_row(state_curr, transitions, a)

In [18]:
transitions = trans_probs_wrapper(A, S)

Filling transitions probabilities for A = 0 (do nothing)...
Filling transitions probabilities for other A...
Transitions done.


In [46]:
# COST FUNCTION

def calc_cost(t, v, r, a):
    if a + r > N_PLANTS:
        return np.inf
    carbontax = C_CO2_INIT * (1.05 ** t)
    cost_fplants = (N_PLANTS - a) * (C_OM_FF*PLANT_SIZE + FF_EMIT*PLANT_SIZE*CAPACITY*HOURS_YR*carbontax)
    # Assume renewable plants cost nothing after construction.
    cost_rplants = a * C_CAP_RES[v]*PLANT_SIZE
    total = cost_rplants + cost_fplants
    return round(total/1e6)

In [39]:
# REWARDS MATRIX

def rewards_wrapper(A, S):
    rewards = np.zeros([S, A])
    print("Filling rewards...")
    fill_rewards(rewards, S)
    print("Rewards done.")
    return rewards

def fill_rewards(rewards, S):
    for a in np.arange(A):
        for s in np.arange(S):
            state = id_to_state[s]
            idx = state_to_id[state]
            # Sanity check for integer id.
            assert(idx == s)
            (t, v, r) = state
            cost = calc_cost(t, v, r, a)
            # Model reward as negative cost.
            rewards[idx][a] = -1 * cost

In [35]:
rewards = rewards_wrapper(A, S)

Filling rewards...
Rewards done.


In [22]:
# HELPER FUNCTIONS

def get_iter_states(n_years, n_techstages, n_plants):
    return it.product(np.arange(n_years+1), np.arange(n_techstages), np.arange(n_plants+1))

def breakdown_state(state):
    (t, v, r) = state
    state_curr = state
    idx_curr = state_to_id[state_curr]
    return (t, v, r), state_curr, idx_curr

def normalize_trans_row(state_curr, transitions, a):
    idx_curr = state_to_id[state_curr]
    transitions[a][idx_curr] = transitions[a][idx_curr] / np.sum(transitions[a][idx_curr])

# Any number of existing renewable plants may fail (at end of year).
def loop_failure(state, transitions, a_actual, a): 
    (t, v, r), state_curr, idx_curr = breakdown_state(state)
    for e in np.arange(r+1):
        prob_fail = binom.pmf(e, r, P_RPLANT_FAIL)
        plants_next = r-e+a_actual
        state_next = (t+1, v, plants_next)
        idx_next = state_to_id[state_next]
        if v < N_TECHSTAGES - 1:
            state_next_v = (t+1, v+1, plants_next)
            idx_next_v = state_to_id[state_next_v]
            # Tech stage may remain the same.
            transitions[a][idx_curr][idx_next] = (1.0-P_ADV_TECHSTAGE) * prob_fail
            # Tech stage may advance (assume only possible to advance by 1).
            transitions[a][idx_curr][idx_next_v] = P_ADV_TECHSTAGE * prob_fail
        else:
            # Tech stage must remain the same.
            transitions[a][idx_curr][idx_next] = prob_fail

In [23]:
# UNIT TESTS

def test_trans_prob(state_curr, state_next, action, expected_prob):
    idx_curr = state_to_id[state_curr]
    idx_next = state_to_id[state_next]
    print(state_curr, " to ", state_next, " with action", action, ":", 
          transitions[action][idx_curr][idx_next], ",", expected_prob)
    assert(transitions[action][idx_curr][idx_next] == expected_prob)

trans_to_test = []
trans_to_test.append(((0, 0, 0), (1, 0, 1), 1, (1.0-P_ADV_TECHSTAGE)))

for sc, sn, a, ep in trans_to_test:
    test_trans_prob(sc, sn, a, ep)

(0, 0, 0)  to  (1, 0, 1)  with action 1 : 0.75 , 0.75


In [50]:
def MDP_wrapper(n_years, n_techstages, n_plants):
    print("Total years:", n_years)
    print("Tech stages:", n_techstages)
    print("Total plants:", n_plants, end="\n")
    S = (n_years+1) * (n_techstages) * (n_plants+1)
    A = n_plants+1
    print("\nInitializing MDP...")
    state_to_id = enumerate_states(N_YEARS, N_TECHSTAGES, N_PLANTS)
    id_to_state = {v: k for k, v in state_to_id.items()}
    transitions = trans_probs_wrapper(A, S)
    rewards = rewards_wrapper(A, S)
    print("REWARDS\n")
    for row, state in zip(rewards, get_iter_states(n_years, n_techstages, n_plants)):
        print(state, ":", row)
    mdp_FH = mtb.mdp.FiniteHorizon(transitions, rewards, DISC_RATE, N_YEARS)
    print("Initialization done.\n\nRunning MDP...")
    mdp_FH.run()
    print("MDP done.\n")
    print("Optimal policy:\nState\t     Time")
    for row, state in zip(mdp_FH.policy, get_iter_states(n_years, n_techstages, n_plants)):
        print(state, ": ", row)
    return mdp_FH

In [51]:
mdp_FH = MDP_wrapper(N_YEARS, N_TECHSTAGES, N_PLANTS)

Total years: 30
Tech stages: 3
Total plants: 5

Initializing MDP...
Filling transitions probabilities for A = 0 (do nothing)...
Filling transitions probabilities for other A...
Transitions done.
Filling rewards...
Rewards done.
REWARDS

(0, 0, 0) : [ -610. -1258. -1907. -2555. -3204. -3852.]
(0, 0, 1) : [ -610. -1258. -1907. -2555. -3204.   -inf]
(0, 0, 2) : [ -610. -1258. -1907. -2555.   -inf   -inf]
(0, 0, 3) : [ -610. -1258. -1907.   -inf   -inf   -inf]
(0, 0, 4) : [ -610. -1258.   -inf   -inf   -inf   -inf]
(0, 0, 5) : [-610.  -inf  -inf  -inf  -inf  -inf]
(0, 1, 0) : [ -610.  -936. -1261. -1587. -1912. -2238.]
(0, 1, 1) : [ -610.  -936. -1261. -1587. -1912.   -inf]
(0, 1, 2) : [ -610.  -936. -1261. -1587.   -inf   -inf]
(0, 1, 3) : [ -610.  -936. -1261.   -inf   -inf   -inf]
(0, 1, 4) : [-610. -936.  -inf  -inf  -inf  -inf]
(0, 1, 5) : [-610.  -inf  -inf  -inf  -inf  -inf]
(0, 2, 0) : [ -610.  -762.  -913. -1065. -1216. -1368.]
(0, 2, 1) : [ -610.  -762.  -913. -1065. -1216.   -in

(23, 2, 4) : [-1446. -1430.   -inf   -inf   -inf   -inf]
(23, 2, 5) : [-1446.   -inf   -inf   -inf   -inf   -inf]
(24, 0, 0) : [-1508. -1977. -2446. -2914. -3383. -3852.]
(24, 0, 1) : [-1508. -1977. -2446. -2914. -3383.   -inf]
(24, 0, 2) : [-1508. -1977. -2446. -2914.   -inf   -inf]
(24, 0, 3) : [-1508. -1977. -2446.   -inf   -inf   -inf]
(24, 0, 4) : [-1508. -1977.   -inf   -inf   -inf   -inf]
(24, 0, 5) : [-1508.   -inf   -inf   -inf   -inf   -inf]
(24, 1, 0) : [-1508. -1654. -1800. -1946. -2092. -2238.]
(24, 1, 1) : [-1508. -1654. -1800. -1946. -2092.   -inf]
(24, 1, 2) : [-1508. -1654. -1800. -1946.   -inf   -inf]
(24, 1, 3) : [-1508. -1654. -1800.   -inf   -inf   -inf]
(24, 1, 4) : [-1508. -1654.   -inf   -inf   -inf   -inf]
(24, 1, 5) : [-1508.   -inf   -inf   -inf   -inf   -inf]
(24, 2, 0) : [-1508. -1480. -1452. -1424. -1396. -1368.]
(24, 2, 1) : [-1508. -1480. -1452. -1424. -1396.   -inf]
(24, 2, 2) : [-1508. -1480. -1452. -1424.   -inf   -inf]
(24, 2, 3) : [-1508. -1480. -14

(6, 2, 2) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(6, 2, 3) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(6, 2, 4) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(6, 2, 5) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(7, 0, 0) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(7, 0, 1) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(7, 0, 2) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(7, 0, 3) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(7, 0, 4) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(7, 0, 5) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(7, 1, 0) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(7, 1, 1) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(7, 1, 2) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(7, 1, 3) :  [0 0 0 0 0 0

(23, 0, 4) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(23, 0, 5) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(23, 1, 0) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(23, 1, 1) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(23, 1, 2) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(23, 1, 3) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(23, 1, 4) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(23, 1, 5) :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(23, 2, 0) :  [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
(23, 2, 1) :  [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
(23, 2, 2) :  [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
(23, 2, 3) :  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
(23, 2, 4) :  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
(23, 2, 5) :

In [ ]:
# Test different combinations of parameters.
    # Time range.
    # Tech stages.
    # Total plants.
    # Starting carbon tax.
    # Carbon tax annual increase.
iter_params = it.product([50], [3], np.arange(5, 11))
MDP_results = {}

blockPrint()

for t, v, r in iter_params:
    mdp_FH = MDP_wrapper(t, v, r)
    MDP_results[(t, v, r)] = mdp_FH.policy

enablePrint()